In [3]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import bqplot as bq
import ipywidgets as widgets
import streamlit as st

In [4]:
df = pd.read_csv('/Users/madisenleshoure/Downloads/BKB_WaterQualityData_2020084.csv')
df

,Site_Id,Unit_Id,Read_Date,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp-Celsius,Air Temp (?F),Time (24:00),Field_Tech,DateVerified,WhoVerified,AirTemp (C),Year
0,Bay,NaN,1/3/1994,1.3,11.7,7.3,0.40,0.40,5.9,8.0,46.40,11:00,NaN,NaN,NaN,8.000000,1994
1,Bay,NaN,1/31/1994,1.5,12.0,7.4,0.20,0.35,3.0,2.6,36.68,11:30,NaN,NaN,NaN,2.600000,1994
2,Bay,NaN,2/7/1994,1.0,10.5,7.2,0.25,0.60,5.9,7.6,45.68,9:45,NaN,NaN,NaN,7.600000,1994
3,Bay,NaN,2/23/1994,1.0,10.1,7.4,0.35,0.50,10.0,2.7,36.86,NaN,NaN,NaN,NaN,2.700000,1994
4,Bay,NaN,2/28/1994,1.0,12.6,7.2,0.20,0.40,1.6,0.0,32.00,10:30,NaN,NaN,NaN,0.000000,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Bay,NaN,10/11/2018,1.9,5.0,7.0,4.00,1.20,25.0,NaN,78.00,09:30,Sue Poe,11/13/2019,Christine Folks,25.555556,2018
2367,Bay,NaN,10/24/2018,0.0,9.0,7.0,0.30,0.60,18.0,NaN,58.00,09:30,Sue Poe,11/13/2019,Christine Folks,14.444444,2018
2368,Bay,NaN,10/28/2018,0.9,2.9,7.0,0.40,0.90,13.0,NaN,49.00,09:20,Sue Poe,11/13/2019,Christine Folks,9.444444,2018
2369,Bay,NaN,11/7/2018,1.7,NaN,7.0,0.45,0.90,20.0,NaN,65.00,09:45,Sue Poe,11/13/2019,Christine Folks,18.333333,2018


In [5]:

df_filtered = df[["Year", "Salinity (ppt)", "Dissolved Oxygen (mg/L)", 
                  "pH (standard units)", "Water Depth (m)"]].dropna()


df_filtered["Decade"] = (df_filtered["Year"] // 10) * 10
df_filtered["Decade_Label"] = df_filtered["Decade"].astype(str) + "–" + (df_filtered["Decade"] + 9).astype(str)


decade_labels = sorted(df_filtered["Decade_Label"].unique())
initial_decade = decade_labels[0]
df_decade = df_filtered[df_filtered["Decade_Label"] == initial_decade]


x_sc = bq.LinearScale()
y_sc = bq.LinearScale()
c_sc = bq.ColorScale(scheme='plasma')
s_sc = bq.LinearScale()


scat = bq.Scatter(
    x=df_decade["Salinity (ppt)"],
    y=df_decade["Dissolved Oxygen (mg/L)"],
    color=df_decade["pH (standard units)"],
    size=df_decade["Water Depth (m)"],
    scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': s_sc},
    default_size=50,
    enable_hover=True)


ax_x = bq.Axis(scale=x_sc, label='Salinity (ppt)')
ax_y = bq.Axis(scale=y_sc, orientation='vertical', label='Dissolved Oxygen (mg/L)')

panzoom = bq.PanZoom(scales={'x': [x_sc], 'y': [y_sc]})


fig = bq.Figure(marks=[scat], axes=[ax_x, ax_y],
                title='Salinity vs Dissolved Oxygen by Decade',
                interaction=panzoom)

dropdown = widgets.Dropdown(options=decade_labels, description="Decade:")

def update_plot(change):
    dec_label = change['new']
    df_new = df_filtered[df_filtered["Decade_Label"] == dec_label]
    scat.x = df_new["Salinity (ppt)"]
    scat.y = df_new["Dissolved Oxygen (mg/L)"]
    scat.color = df_new["pH (standard units)"]
    scat.size = df_new["Water Depth (m)"]


    x_range = [df_new["Salinity (ppt)"].min(), df_new["Salinity (ppt)"].max()]
    y_range = [df_new["Dissolved Oxygen (mg/L)"].min(), df_new["Dissolved Oxygen (mg/L)"].max()]
    x_sc.min, x_sc.max = x_range
    y_sc.min, y_sc.max = y_range

dropdown.observe(update_plot, names='value')


widgets.VBox([dropdown, fig])

In [6]:
from bqplot.colorschemes import CATEGORY10

df['Read_Date'] = pd.to_datetime(df['Read_Date'], errors='coerce')

y_var = "Water Temp (?C)"


plot_df = df[["Read_Date", "Site_Id", y_var]].dropna()
plot_df = plot_df[plot_df["Read_Date"].notnull()]
site_ids = sorted(plot_df["Site_Id"].unique())
colors = CATEGORY10[:len(site_ids)]
site_color_map = dict(zip(site_ids, colors))

x_sc = bq.DateScale()
y_sc = bq.LinearScale()


line = bq.Lines(
    x=[],
    y=[],
    labels=[],
    display_legend=True,
    stroke_width=2,
    scales={'x': x_sc, 'y': y_sc},
    tooltip=bq.Tooltip(fields=['labels'], labels=['Site']),
    enable_hover=True
)


x_ax = bq.Axis(label="Read Date", scale=x_sc)
y_ax = bq.Axis(label=y_var, scale=y_sc, orientation="vertical")


panzoom = bq.PanZoom(scales={'x': [x_sc], 'y': [y_sc]})


fig = bq.Figure(
    marks=[line],
    axes=[x_ax, y_ax],
    interaction=panzoom,
    title=f"{y_var} over Time"
)


dropdown = widgets.Dropdown(
    options=site_ids,
    description='Site_Id:',
    value=site_ids[0]
)


def update_line(site):
    group = plot_df[plot_df["Site_Id"] == site].sort_values("Read_Date")
    line.x = group["Read_Date"]
    line.y = group[y_var]
    line.labels = [site] * len(group)
    line.colors = [site_color_map[site]]

dropdown.observe(lambda change: update_line(change['new']) if change['name'] == 'value' else None, names='value')


update_line(site_ids[0])


widgets.VBox([dropdown, fig])